In [29]:
import ipynb
import torch
from ipynb.fs.full.data_handler import load_subject, extract_labels, extract_action_interval, split_data, split_info, to_device, get_innerspeech
from ipynb.fs.full.train_model import train_model, accuracy_check

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

data, description = load_subject(1)
data, description = get_innerspeech(data, description)
labels = extract_labels(description)
data_interval = extract_action_interval(data)

train_data, val_data, test_data, train_labels, val_labels, test_labels = split_data(data_interval, labels)

train_data, val_data, test_data, train_labels, val_labels, test_labels = to_device(train_data, val_data, test_data, train_labels, val_labels, test_labels, device)
split_info(train_data, val_data, test_data, train_labels, val_labels, test_labels)

[39, 39, 40, 40] 
 [6, 6, 5, 5] 
 [5, 5, 5, 5]
24.68354430379747 %  24.68354430379747 %  25.31645569620253 %  25.31645569620253 % 
27.27272727272727 %  27.27272727272727 %  22.727272727272727 %  22.727272727272727 % 
25.0 %  25.0 %  25.0 %  25.0 % 


# Deep CNN
Implementation of the deep CNN structure from (Schirrmeister et. al.) for the use on "Thinking out loud" dataset.

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim

class DeepCNN(nn.Module):
    def __init__(self, interval = "full", dropout = 0.5):
        super(DeepCNN, self).__init__()
        first_channels = 25
        second_channels = 50
        third_channels = 100
        fourth_channels = 200

        ### Block 1
        # Temporal convolution
        self.tempconv = nn.Conv2d(in_channels = 1, out_channels = first_channels, kernel_size = (1, 10), padding = 0, bias = False)
        # Spatial convolution
        self.spatconv = nn.Conv2d(in_channels = first_channels, out_channels = first_channels, kernel_size=(128,1), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm1 = nn.BatchNorm2d(first_channels, False)
        # ELU
        self.elu = nn.ELU()
        # Dropout
        self.dropout = nn.Dropout(dropout)
        # Max pooling
        self.maxpool = nn.MaxPool2d(kernel_size = (1,3), stride = (1,3))

        ### Block 2
        self.conv2 = nn.Conv2d(in_channels = first_channels, out_channels = second_channels, kernel_size = (1,10), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm2 = nn.BatchNorm2d(second_channels, False)

        ### Block 3
        self.conv3 = nn.Conv2d(in_channels = second_channels, out_channels = third_channels, kernel_size = (1,10), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm3 = nn.BatchNorm2d(third_channels, False)

        ### Block 4
        self.conv4 = nn.Conv2d(in_channels = third_channels, out_channels = fourth_channels, kernel_size = (1,10), padding = 0, bias = False)
        # Batch normalization
        self.batchnorm4 = nn.BatchNorm2d(fourth_channels, False)

        # Classifier
        if interval == "action":
            self.classifier = nn.Linear(600,4, bias = False)
        elif interval == "full":
            self.classifier = nn.Linear(1800,4, bias = False)
        # Softmax
        self.softmax = nn.LogSoftmax(dim = 1)

        
    def forward(self, x):
        # Block 1
        res = self.tempconv(x)
        res = self.spatconv(res)
        res = self.batchnorm1(res)
        res = self.elu(res)
        res = self.maxpool(res)
        res = self.dropout(res)
        # Block 2
        res = self.conv2(res)
        res = self.batchnorm2(res)
        res = self.elu(res)
        res = self.maxpool(res)
        # Block 3
        res = self.conv3(res)
        res = self.batchnorm3(res)
        res = self.elu(res)
        res = self.maxpool(res)
        # Block 4
        res = self.conv4(res)
        res = self.batchnorm4(res)
        res = self.elu(res)
        res = self.maxpool(res)
        # Classifier
        res = torch.flatten(res, start_dim=1)
        res = self.classifier(res)
        res = self.softmax(res)
        return res

In [31]:
network = DeepCNN(interval = "action").float().to(device)
accuracy_check(network, train_data, train_labels)

print("####### TRAINING #######")
op = optim.Adam(params = network.parameters(), lr = 0.0001)
lossf = nn.NLLLoss()
train_model(network, train_data = train_data, train_labels = train_labels, val_data = val_data, val_labels = val_labels, epochs = 50, batch_size = 10, loss_func = lossf, optimizer = op)

accuracy_check(network, train_data, train_labels)

ACCURACY: 0.2468354430379747
####### TRAINING #######
Epoch	 train loss	 validation loss
0 	  1.5203728993733725 	 1.4444648027420044
1 	  1.2355884552001952 	 1.5502461194992065
2 	  1.141515604654948 	 1.669081211090088
3 	  1.0590504964192708 	 1.7558993101119995
4 	  1.0026379267374674 	 1.8320422172546387
5 	  0.8916405359903972 	 1.8749492168426514
6 	  0.7853562672932942 	 1.9128414392471313
7 	  0.7399758021036784 	 1.959327220916748
8 	  0.669963264465332 	 1.9809632301330566
9 	  0.5819945017496745 	 2.0261189937591553
10 	  0.5008022626241048 	 2.159670829772949
11 	  0.46645221710205076 	 2.2846415042877197
12 	  0.3815501848856608 	 2.445828437805176
13 	  0.3621055603027344 	 2.431354284286499
14 	  0.340928586324056 	 2.2949140071868896
15 	  0.27345345815022787 	 2.2113144397735596
16 	  0.24733529090881348 	 2.0963311195373535
17 	  0.2222481409708659 	 2.205003499984741
18 	  0.19813793500264484 	 2.2006590366363525
19 	  0.19887890815734863 	 2.3128812313079834
20 	 

In [32]:
accuracy_check(network, train_data, train_labels)
accuracy_check(network, val_data, val_labels)
accuracy_check(network, test_data, test_labels)


ACCURACY: 0.7974683544303798
ACCURACY: 0.2727272727272727
ACCURACY: 0.2
